In [2]:
import cvxpy as cp
import numpy as np

https://www.cvxpy.org/


$$ Max: \sum_{t=1}^T(5x_{1t} ​+10 x_{2t}​+(10−x_{1t}​−2x_{2t}​)+2(10−3x_{2t}​)+3(10−x_{1​t}))  \\
st: 
x_{1t} + 2x_{2t} \leq 10 \\
3x_{2t} \leq 10 \\
x_{1t} \leq 10 \\
5x_{1​t}+10x_{2t}​​​+(10-x_{1t}-2x_{2t})+2(10−3x_{2t}​)+3(10−x_{1t}​) \geq 70\\
x_1, x_2 \geq 0$$

In [20]:
T = 1
x1 = cp.Variable(T, nonneg=True)
x2 = cp.Variable(T, nonneg=True)
objective = cp.Maximize(cp.sum(5*x1 + 10*x2 + (10 - x1 - 2*x2) + 2*(10 - 3*x2) + 3*(10 - x1)))

constraints = [
    x1 + 2*x2 <= 10,
    3*x2 <= 10,
    x1 <= 10,
    5*x1 + 10*x2 + (10 - x1 - 2*x2) + 2*(10 - 3*x2) + 3*(10 - x1) >= 70
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL:
    print("Optimal values:")
    print("Total points earned:", problem.value)
    print("Amount of tools produced:", x1.value)
    print("Amount of housing produced:", x2.value)
    print("Amount of wood left:", 10-(x1.value + 2*x2.value))
    print("Amount of stone left:", 10-(3*x2.value))
    print("Amount of iron left:", 10-x1.value)

Optimal solution found!
Total points earned: 70.00000000180277
Amount of tools produced: [5.77139996]
Amount of housing produced: [2.11430002]
Amount of wood left: [-1.80276949e-09]
Amount of stone left: [3.65709994]
Amount of iron left: [4.22860004]



$$ Max: \sum_{t=1}^T(x_1 + 2x_2 + 3x_3 + 5x_4 + 10x_5)  \\
st: 
x_1 + 3x_2 + 2x_3 + x_4 + 2(x_5) \leq 100 \\
x_1 - x_4 - 2x_5 \geq 0 \\
x_2 - 3 \cdot x_5 \geq 0 \\
x_3 - x_1 \geq 0 \\
x_3 \leq 10 \\
x_1, x_2, x_3, x_4,x_5 \geq 0$$

In [28]:

T = 1 
x1 = cp.Variable(T, nonneg=True)  # wood used
x2 = cp.Variable(T, nonneg=True)  # stone used
x3 = cp.Variable(T, nonneg=True)  # iron extracted
x4 = cp.Variable(T, nonneg=True)  # tools produced
x5 = cp.Variable(T, nonneg=True)  # housing produced

objective = cp.Maximize(cp.sum(x1 + 2*x2 + 3*x3 + 5*x4 + 10*x5))
constraints = [
    x1 + 3*x2 + 2*x3 + x4 + 2*x5 <= 100,
    x1 - x4 - 2*x5 >=0,
    x2 - 3*x5 >=0,
    x3 - x1 >=0,
    x3 <= 10
]

problem = cp.Problem(objective, constraints)
problem.solve()

# Output the results
print("Optimal values:")
print("Wood used:", x1.value)
print("Stone used:", x2.value)
print("Iron extracted:", x3.value)
print("Tools produced:", x4.value)
print("Housing produced:", x5.value)
print("Optimal objective value:", problem.value)


Optimal values:
Wood used: [10.]
Stone used: [19.99999999]
Iron extracted: [10.]
Tools produced: [2.98274144]
Housing produced: [3.50862928]
Optimal objective value: 129.9999999949219
